# Limpieza de dataset de la Encuesta Intercensal 2015
## 1 . Introduccion
Para la construcción de indicadores de la Plataforma de Conocimiento de Ciudades Sustentables se han considerado los siguientes datos disponibles desde la Encuesta Intercensal del INEGI:

ID |Descripción
---|:----------
P0101|Porcentaje de viviendas con agua entubada
P0102|Porcentaje de viviendas que cuentan con descarga a una red de alcantarillado.
P0402|Viviendas que aprovechan energía solar
P0403|Viviendas con drenaje
P0404|Viviendas con piso de tierra
P0603|Viviendas particulares habitadas con calentador de agua (boiler)
P0611|Viviendas que utilizan leña o carbón para cocinar
P0612|Viviendas que utilizan leña o carbón para cocinar, que disponen de estufa o fogón con chimenea
P0613|Viviendas habitadas que utilizan gas para cocinar
P1004|Forma de eliminación de residuos
P1010|Porcentaje de viviendas  de reutilización de residuos
P1011|Porcentaje de viviendas que separan sus residuos en orgánicos e inorgánicos

En este documento se describen los pasos llevados a cabo para estandarizar los datos disponibles desde la encuesta intercensal y utilizarlos para la construcción de parámetros. Las limpieza se realiza utilizando Python 3.

## 2 . Definiciones

**PCCS** : Plataforma de Conocimiento de Ciudades Sustentables

**Dataset** : Conjunto de datos que tratan acerca de un tema, usualmente ordenados para facilitar su lectura por un ser humano, 
que por lo general requieren ser procesados para facilitar su lectura por sistemas informáticos. 

**Dataframe** : Estructura bidimensional de datos, estructurada en filas que contienen casos y columnas que contienen variables.

## 3 . Descarga de Datos

In [194]:
# Librerias utilizadas
import pandas as pd
import sys
import urllib
import os
import numpy as np

In [2]:
# Configuracion del sistema
print('Python {} on {}'.format(sys.version, sys.platform))
print('Pandas version: {}'.format(pd.__version__))
import platform; print('Running on {} {}'.format(platform.system(), platform.release()))

Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)] on win32
Pandas version: 0.20.1
Running on Windows 8.1


La descarga de datos se realiza desde el sitio Beta de INEGI. Los datos de la Encuesta Intercensal 2015 se encuentran en http://www.beta.inegi.org.mx/proyectos/enchogares/especiales/intercensal/

Existen tres maneras de descargar la información:

1. **Datos para la República Mexicana**, con la ventaja de que es un solo archivo con variables procesadas y con la desventaja de que su nivel de desagregación es estatal.
2. **Datos estatales**, con la ventaja de que cuentan con desagregacion a nivel municipal con variables interpretadas y con la desventaja de que la información está fragmentada en muchos archivos pues hay un archivo por variable por estado.
3. **Microdatos**, con la ventaja de que contienen toda la información del Proyecto en pocos archivos y con la desventaja de que tienen que interpretarse antes de obtener valores útiles para la PCCS.

La manera más conveniente es descargar los datos estatales, pues la primera no entregaría datos relevantes para la construccion de indicadores de la PCCS y la segunda requeriría dedicar una gran cantidad de tiempo y esfuerzo para recrear la interpretacion realizada por INEGI. 
Todos los indicadores que se utilizarán para la construccion de la PCCS se encuentran en la encuesta de Vivienda, por lo que únicamente se descargará el paquete de datos de esta encuesta

In [26]:
# LIGAS PARA DESCARGA DE ARCHIVOS
# Las ligas para descarga tienen una raiz URL común que cambia 
# dependiendo del indicador y estado que se busque descargar 
url = r'http://www.beta.inegi.org.mx/contenidos/Proyectos/enchogares/especiales/intercensal/2015/tabulados/'
indicador = r'14_vivienda_'
raiz = url+indicador
links = {
    '01' : raiz+'ags.xls',
    '02' : raiz+'bc.xls',
    '03' : raiz+'bcs.xls',
    '04' : raiz+'cam.xls',
    '05' : raiz+'coah.xls',
    '06' : raiz+'col.xls',
    '07' : raiz+'chis.xls',
    '08' : raiz+'chih.xls',
    '09' : raiz+'cdmx.xls',
    '10' : raiz+'dgo.xls',
    '11' : raiz+'gto.xls',
    '12' : raiz+'gro.xls',
    '13' : raiz+'hgo.xls',
    '14' : raiz+'jal.xls',
    '15' : raiz+'mex.xls',
    '16' : raiz+'mich.xls',
    '17' : raiz+'mor.xls',
    '18' : raiz+'nay.xls',
    '19' : raiz+'nl.xls',
    '20' : raiz+'oax.xls',
    '21' : raiz+'pue.xls',
    '22' : raiz+'qro.xls',
    '23' : raiz+'qroo.xls',
    '24' : raiz+'slp.xls',
    '25' : raiz+'sin.xls',
    '26' : raiz+'son.xls',
    '27' : raiz+'tab.xls',
    '28' : raiz+'tamps.xlsz',
    '29' : raiz+'tlax.xls',
    '30' : raiz+'ver.xls',
    '31' : raiz+'yuc.xls',
    '32' : raiz+'zac.xls'
}

Las ligas quedan almacenadas en un diccionario de python en el que "key" = Clave Geoestadística Estatal; y value = liga para descarga. Por ejemplo, '09' es la clave geoestadística para la Ciudad de México. Si al diccionario links le solicitamos el valor de la key '09', nos regresa la liga para descargar los indicadores de vivienda de la Ciudad de México, como se muestra a continuación:

In [25]:
print(links['09'])

http://www.beta.inegi.org.mx/contenidos/Proyectos/enchogares/especiales/intercensal/2015/tabulados/14_vivienda_cdmx.xls


Con el diccionario de ligas ya es posible descargar los archivos en una carpeta local para poder procesarlos.

In [70]:
# Descarga de archivos a carpeta local
destino = r'D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda'
archivos = {}   # Diccionario para guardar memoria de descarga
for k,v in links.items():
    archivo_local = destino + r'\{}.xls'.format(k)
    if os.path.isfile(archivo_local):
        print('Ya existe el archivo: {}'.format(archivo_local))
        archivos[k] = archivo_local
    else:
        print('Descargando {} ... ... ... ... ... '.format(archivo_local))
        urllib.request.urlretrieve(v, archivo_local) #
        archivos[k] = archivo_local
        print('se descargó {}'.format(archivo_local))

Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\01.xls
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\02.xls
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\03.xls
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\04.xls
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\05.xls
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\06.xls
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\07.xls
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\08.xls
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\09.xls
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\10.xls
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivi

Cada archivo tiene la misma estructura y contiene los datos de vivienda de 2015 levantados en la encuesta intercensal. La primera hoja, 'Índice', incluye un listado de las hojas y datos que contiene cada libro. Este índice se tomará como referencia para la minería de datos:

In [29]:
pd.options.display.max_colwidth = 150
df = pd.read_excel(archivos['01'],
              sheetname = 'Índice',
              skiprows = 6,
              usecols = ['Tabulado', 'Título'],
              dtype = {'Tabulado' : 'str'},
              ).set_index('Tabulado')
df

,Título
Tabulado,
01,Estimadores de las viviendas particulares habitadas y su distribución porcentual según clase de vivienda particular por municipio
02,Estimadores de las viviendas particulares habitadas y su distribución porcentual según material en pisos por municipio
03,Estimadores de las viviendas particulares habitadas y su distribución porcentual según resistencia de los materiales en techos por municipio
04,Estimadores de las viviendas particulares habitadas y su distribución porcentual según resistencia de los materiales en paredes por municipio
05,Estimadores de las viviendas particulares habitadas y su distribución porcentual según número de cuartos por municipio
06,Estimadores de las viviendas particulares habitadas y su distribución porcentual según número de dormitorios por municipio
07,Estimadores de las viviendas particulares habitadas en las que sus ocupantes cocinan y su distribución porcentual según lugar de la vivienda o pre...
08,Estimadores de las viviendas particulares habitadas y su distribución porcentual según combustible utilizado para cocinar por municipio
09,Estimadores de las viviendas particulares habitadas en las que sus ocupantes utilizan leña o carbón para cocinar y su distribución porcentual segú...


La columna 'Tabulado' contiene el nombre de la hoja, mientras que 'Titulo' describe los datos de la misma. Para la construcción de parámetros de la PCCS se utilizarán las siguientes hojas:

HOJA | PARAMETRO | DESCRIPCION
----|----------|:-----------
25|P0101|Porcentaje de viviendas con agua entubada
26|P0102|Porcentaje de viviendas que cuentan con descarga a una red de alcantarillado.
26|P0403|Viviendas con drenaje
02|P0404|Viviendas con piso de tierra
23|P0603|Viviendas particulares habitadas con calentador de agua (boiler)
08|P0611|Viviendas que utilizan leña o carbón para cocinar
09|P0612|Viviendas que utilizan leña o carbón para cocinar, que disponen de estufa o fogón con chimenea
08|P0613|Viviendas habitadas que utilizan gas para cocinar
19|P1004|Forma de eliminación de residuos
21|P1010|Porcentaje de viviendas  de reutilización de residuos
20|P1011|Porcentaje de viviendas que separan sus residuos en orgánicos e inorgánicos

Los siguientes parámetros se pueden obtener desde otras fuentes, pero se incluirán en esta minería por encontrarse también disponibles para 2015 en este dataset.

HOJA  | PARAMETRO  | DESCRIPCION
----  | ---------- | :-----------
23 | P0604 | Viviendas particulares habitadas con calentador solar
23 | P0605 | Viviendas particulares habitadas con panel fotovoltaico

## 4 . Estandarización de Dataset
A partir de las hojas identificadas y asociadas con parámetros, es necesario crear un dataframe estándar que sea de fácil lectura para sistemas informáticos y permita la creación de parámetros para la PCCS. Cada hoja de las descritas anteriormente tiene un acomodo distinto de las variables y requiere un proceso diferente, aunque la secuencia general para todas las hojas será la siguiente:
    
    1. Crear una función que sirva para extraer los datos de una hoja "tipo"
    2. Correr la función sobre cada archivo de excel y juntar los datos recopilados en un solo DataFrame
    3. Limpieza final al dataframe y guardado


**HOJA 02: Estimadores de las viviendas particulares habitadas y su distribución porcentual según material en pisos por tamaño de localidad**

1 . Funcion para extraer datos de hoja tipo

In [69]:
# Funcion para extraer datos de hoja tipo
# La funcion espera los siguientes valores:
# --- entidad: clave geoestadistica de entidad de 2 digitos
# --- ruta: ruta al archivo de excel que contiene la información
# --- hoja: numero de hoja dentro del archivo de excel que se pretende procesar
# --- colnames: nombres para las columnas de datos (Las columnas en los archivos de este 
#               dataset requieren ser nombradas manualmente por la configuración de los
#               encabezados en los archivo fuente)
# --- skip: El numero de renglones en la hoja que el script tiene que ignorar para encontrar
#           el renglon de encabezados.

def cargahoja(entidad, ruta, hoja, colnames, skip):
    # Abre el archivo de excel
    raw_data = pd.read_excel(ruta,
                              sheetname=hoja,
                              skiprows=skip).dropna()
    # renombra las columnas
    raw_data.columns = colnames

    # Obten Unicamente las filas con valores estimativos
    raw_data = raw_data[raw_data['Estimador'] == 'Valor']

    # Crea la columna CVE_MUN 
    raw_data['CVE_ENT'] = entidad
    raw_data['ID_MUN'] = raw_data.Municipio.str.split(' ', n=1).apply(lambda x: x[0])
    raw_data['CVE_MUN'] = raw_data['CVE_ENT'].map(str) + raw_data['ID_MUN']

    # Borra columnas con informacion irrelevante
    del (raw_data['CVE_ENT'])
    del (raw_data['ID_MUN'])
    del (raw_data['Entidad federativa'])
    del (raw_data['Estimador'])
    raw_data.set_index('CVE_MUN', inplace=True)
    return raw_data

2 . Correr funcion sobre todos los archivos de excel 

In [31]:
# correr funcion sobre todos los archivos
colnames = ['Entidad federativa', 
            'Municipio',
            'Estimador',
            'Viviendas particulares habitadas', 
            'Pisos_Tierra', 
            'Pisos_Cemento o firme', 
            'Pisos_Mosaico, madera u otro recubrimiento', 
            'Pisos_No especificado']

DatosPiso = {}
for k,v in archivos.items():
    print('Procesando {}'.format(v))
    hoja = cargahoja(k, v, '02', colnames, 7)
    DatosPiso[k] = hoja

Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\01.xls
Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\02.xls
Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\03.xls
Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\04.xls
Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\05.xls
Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\06.xls
Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\07.xls
Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\08.xls
Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\09.xls
Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\10.xls
Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\11.xls
Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\12.xls
Procesando D:\PCCS\00_RawDat

Los datos fueron guardados como diccionario de Python, es necesario convertirlos en un DataFrame unico antes de hacer la limpieza final.

In [105]:
PisosDF = pd.DataFrame()
for k,v in DatosPiso.items():
    PisosDF = PisosDF.append(v)

3 . Limpieza final del Dataframe 'Pisos': El dataframe está casi listo para ser utilizado en la construcción de indicadores, únicamente hace falta quitar algunas lineas de "basura" que tienen los datos de totales por Municipio.

In [46]:
PisosDF = PisosDF[PisosDF['Municipio'] != 'Total']

In [115]:
PisosDF.describe()

,Viviendas particulares habitadas,Pisos_Tierra,Pisos_Cemento o firme,"Pisos_Mosaico, madera u otro recubrimiento",Pisos_No especificado
count,2.489000e+03,2489.000000,2489.000000,2489.000000,2489.000000
mean,2.565276e+04,8.351254,71.418885,19.447967,0.781893
std,1.492035e+05,8.920021,17.122480,19.676034,6.620830
min,3.100000e+01,0.000000,0.000000,0.000000,0.000000
25%,1.139000e+03,1.954625,64.558108,4.139324,0.122063
50%,3.559000e+03,5.245374,76.779113,12.323689,0.251073
75%,9.216000e+03,11.400000,83.438914,28.543307,0.466744
max,4.166570e+06,66.559203,97.297297,94.527410,100.000000


**HOJAS 08, 09, 19, 20, 21 23, 25, 26**

Como se mencionó antes, todas las hojas siguen un proceso similar para la extraccion de datos, con ligeras variaciones. 

1 . Funcion para extraer datos de hoja tipo

Para el resto de los archivos reutilizaremos la función "cargahoja" definida anteriormente

2 . Correr función sobre archivos de excel

Para que la función "cargahoja" pueda iterar de manera adecuada sobre todos los archivos, es necesario especificar cuales son las variacione para cada archivo que la función va a leer. Las principales variaciones son los nombres de las columnas y la ubicación de los encabezados en cada hoja, por lo que los datos de cada hoja pueden extraerse de manera automatizada una vez que se identifique cómo va a tratar el script a cada una de las variaciones.

--- 2.1 . A continuación se definen las columnas para cada hoja:

In [124]:
# Se define un diccionario con la siguiente sintaxis: 'NUMERO DE HOJA' : [LISTA DE COLUMNAS]

dicthojas = {
    '08' : [   # Combustible utilizado para cocinar
        'Entidad federativa',
        'Municipio',
        'Estimador',
        'Viviendas particulares habitadas',
        'Cocina_con_Lena o carbon',
        'Cocina_con_Gas',
        'Cocina_con_Electricidad',
        'Cocina_con_Otro_Combustible',
        'Cocina_con_Los_ocupantes_no_cocinan',
        'Cocina_con_no_especificado'
    ], 
    '09' : [   # Utilizan leña o carbón para cocinar y distribucion porcentual segun disponibilidad de estufa o fogon 
        'Entidad federativa',
        'Municipio',
        'Estimador',
        'Viviendas particulares habitadas en las que sus ocupantes utilizan leña o carbon para cocinar',
        'Dispone_de_estufa_o_fogon_con_chimenea',
        'No dispone_de_estufa_o_fogon_con_chimenea',
        'Estufa_o_fogon_no_especificado'
    ],
    '19' : [   # Forma de eliminación de residuos
        'Entidad federativa',
        'Municipio',
        'Estimador',
        'Viviendas particulares habitadas',
        'Entregan_residuos_a_servicio_publico_de_recoleccion',
        'Tiran_residuos_en_basurero_publico_colocan_en_contenedor_o_deposito',
        'Queman_residuos',
        'Entierran_residuos_o_tiran_en_otro_lugar',
        'Eliminacion_de_residuos_no_especificado',
    ],
    '20' : [   # Viviendas que entregan sus residuos al servicio publico y distribucion porcentual por condición de separacion
        'Entidad federativa',
        'Municipio',
        'Estimador',
        'Viviendas particulares habitadas en las que entregan los residuos al servicio publico',
        'Separan_organicos_inorganicos',
        'No_separan_organicos_inorganicos',
        'Separan_residuos_No_especificado'
    ],
    '21' : [   # Separación y reutilización de residuos
        'Entidad federativa',
        'Municipio',
        'Forma de reutilizacion de residuos',
        'Estimador',
        'Viviendas particulares habitadas',
        'Reutilizan_residuos',
        'No_reutilizan_residuos',
        'No_especificado_reutilizan_residuos',
    ], 
    '23' : [   # Disponibilidad y tipo de equipamiento
        'Entidad federativa',
        'Municipio',
        'Tipo de equipamiento',
        'Estimador',
        'Viviendas particulares habitadas',
        'Dispone_de_Equipamiento',
        'No_dispone_de_Equipamiento',
        'No_especificado_dispone_de_Equipamiento'
    ],
    '25' : [   # Disponibilidad de agua entubada y fuente de abastecimiento
        'Entidad federativa',
        'Municipio',
        'Estimador',
        'Viviendas particulares que disponen de agua entubada',
        'Agua_entubada_de_Servicio_Publico',
        'Agua_entubada_de_Pozo_comunitario',
        'Agua_entubada_de_Pozo_particular',
        'Agua_entubada_de_Pipa',
        'Agua_entubada_de_Otra_Vivienda',
        'Agua_entubada_de_Otro_lugar',
        'Agua_entubada_de_No_especificado'
    ],
    '26' : [   # Disponibilidad de drenaje y lugar de desalojo
        'Entidad federativa',
        'Municipio',
        'Estimador',
        'Viviendas particulares habitadas',
        'Drenaje_Total',
        'Drenaje_desaloja_a_Red_publica',
        'Drenaje_desaloja_a_Fosa_Septica_o_Tanque_Septico',
        'Drenaje_desaloja_a_Barranca_o_Grieta',
        'Drenaje_desaloja_a_Rio, lago o mar',
        'No_Dispone_de_drenaje',
        'Dispone_drenaje_No_especificado',
    ]
}

--- 2.2 . Además de definir las columnas, es necesario definir cuántos renglones tiene que ignorar el script antes de encontrar los encabezados. Estos renglones se definen a continuación en un diccionario con :

In [151]:
skiprows = {
    '08' : 7,   # Combustible utilizado para cocinar
    '09' : 7,   # Utilizan leña o carbón para cocinar y distribucion porcentual segun disponibilidad de estufa o fogon 
    '19' : 7,   # Forma de eliminación de residuos
    '20' : 8,   # Viviendas que entregan sus residuos al servicio publico y distribucion porcentual por condición de separacion
    '21' : 7,   # Separación y reutilización de residuos
    '23' : 7,   # Disponibilidad y tipo de equipamiento
    '25' : 7,   # Disponibilidad de agua entubada y fuente de abastecimiento
    '26' : 8,   # Disponibilidad de drenaje y lugar de desalojo
}

2 . Correr funcion sobre todos los archivos de excel

Una vez que se tienen definidos los tratamientos que el script dará a cada variación, se extraen los datos por medio de una función iterativa hacia un diccionario de Python.

In [152]:
HojasDatos = {}
for estado, archivo in archivos.items():
    print('Procesando {}'.format(archivo))
    hojas = {}
    for hoja, columnas in dicthojas.items():
        print('---Procesando hoja {}'.format(hoja))
        dataset = cargahoja(estado, archivo, hoja, columnas, skiprows[hoja])
        if hoja not in HojasDatos.keys():
            HojasDatos[hoja] = {}
        HojasDatos[hoja][estado] = dataset

Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\01.xls
---Procesando hoja 08
---Procesando hoja 09
---Procesando hoja 19
---Procesando hoja 20
---Procesando hoja 21
---Procesando hoja 23
---Procesando hoja 25
---Procesando hoja 26
Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\02.xls
---Procesando hoja 08
---Procesando hoja 09
---Procesando hoja 19
---Procesando hoja 20
---Procesando hoja 21
---Procesando hoja 23
---Procesando hoja 25
---Procesando hoja 26
Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\03.xls
---Procesando hoja 08
---Procesando hoja 09
---Procesando hoja 19
---Procesando hoja 20
---Procesando hoja 21
---Procesando hoja 23
---Procesando hoja 25
---Procesando hoja 26
Procesando D:\PCCS\00_RawData\01_CSV\Intercensal2015\estatal\14. Vivienda\04.xls
---Procesando hoja 08
---Procesando hoja 09
---Procesando hoja 19
---Procesando hoja 20
---Procesando hoja 21
---Procesando hoja 23
---Procesando ho

---Procesando hoja 26


El diccionario resultante contiene los datos de cada hoja clasificados por estado. Sin embargo, la estructura de diccionarios aun requiere ser procesada para obtener dataframes estandar.

In [176]:
# Procesado de diccionarios para obtener datasets estándar
DSstandar = {}
for hoja, estado in HojasDatos.items():
    print('Procesando hoja {}'.format(hoja))
    tempDS = pd.DataFrame()
    for cve_edo, datos in estado.items():
        tempDS = tempDS.append(datos)
        print('---Se agregó CVE_EDO {} a dataframe estandar'.format(cve_edo))
    DSstandar[hoja] = tempDS

Procesando hoja 08
---Se agregó CVE_EDO 01 a dataframe estandar
---Se agregó CVE_EDO 02 a dataframe estandar
---Se agregó CVE_EDO 03 a dataframe estandar
---Se agregó CVE_EDO 04 a dataframe estandar
---Se agregó CVE_EDO 05 a dataframe estandar
---Se agregó CVE_EDO 06 a dataframe estandar
---Se agregó CVE_EDO 07 a dataframe estandar
---Se agregó CVE_EDO 08 a dataframe estandar
---Se agregó CVE_EDO 09 a dataframe estandar
---Se agregó CVE_EDO 10 a dataframe estandar
---Se agregó CVE_EDO 11 a dataframe estandar
---Se agregó CVE_EDO 12 a dataframe estandar
---Se agregó CVE_EDO 13 a dataframe estandar
---Se agregó CVE_EDO 14 a dataframe estandar
---Se agregó CVE_EDO 15 a dataframe estandar
---Se agregó CVE_EDO 16 a dataframe estandar
---Se agregó CVE_EDO 17 a dataframe estandar
---Se agregó CVE_EDO 18 a dataframe estandar
---Se agregó CVE_EDO 19 a dataframe estandar
---Se agregó CVE_EDO 20 a dataframe estandar
---Se agregó CVE_EDO 21 a dataframe estandar
---Se agregó CVE_EDO 22 a dataframe 

---Se agregó CVE_EDO 05 a dataframe estandar
---Se agregó CVE_EDO 06 a dataframe estandar
---Se agregó CVE_EDO 07 a dataframe estandar
---Se agregó CVE_EDO 08 a dataframe estandar
---Se agregó CVE_EDO 09 a dataframe estandar
---Se agregó CVE_EDO 10 a dataframe estandar
---Se agregó CVE_EDO 11 a dataframe estandar
---Se agregó CVE_EDO 12 a dataframe estandar
---Se agregó CVE_EDO 13 a dataframe estandar
---Se agregó CVE_EDO 14 a dataframe estandar
---Se agregó CVE_EDO 15 a dataframe estandar
---Se agregó CVE_EDO 16 a dataframe estandar
---Se agregó CVE_EDO 17 a dataframe estandar
---Se agregó CVE_EDO 18 a dataframe estandar
---Se agregó CVE_EDO 19 a dataframe estandar
---Se agregó CVE_EDO 20 a dataframe estandar
---Se agregó CVE_EDO 21 a dataframe estandar
---Se agregó CVE_EDO 22 a dataframe estandar
---Se agregó CVE_EDO 23 a dataframe estandar
---Se agregó CVE_EDO 24 a dataframe estandar
---Se agregó CVE_EDO 25 a dataframe estandar
---Se agregó CVE_EDO 26 a dataframe estandar
---Se agre

3 . Limpieza final de los Dataframe: Antes de habilitar los dataframes para ser utilizados en la construcción de indicadores, hace falta quitar algunas lineas de "basura" que contienen datos de totales por Municipio.

In [178]:
for hoja in DSstandar.keys():
    temphoja = DSstandar[hoja]
    temphoja = temphoja[temphoja['Municipio'] != 'Total'] 
    DSstandar[hoja] = temphoja

In [181]:
DSstandar['08']

,Municipio,Viviendas particulares habitadas,Cocina_con_Lena o carbon,Cocina_con_Gas,Cocina_con_Electricidad,Cocina_con_Otro_Combustible,Cocina_con_Los_ocupantes_no_cocinan,Cocina_con_no_especificado
CVE_MUN,,,,,,,,
01001,001 Aguascalientes,230292.0,0.794209,97.578292,0.800288,0.021277,0.669585,0.136349
01002,002 Asientos,10787.0,6.554186,91.758598,0.268842,0.000000,1.205154,0.213220
01003,003 Calvillo,14166.0,8.245094,90.237188,0.423549,0.021177,1.023578,0.049414
01004,004 Cosío,3519.0,4.205740,94.344984,0.113669,0.056834,0.966184,0.312589
01010,010 El Llano,4876.0,5.598852,91.981132,0.328138,0.000000,2.009844,0.082034
01005,005 Jesús María,28875.0,1.381818,97.011255,0.540260,0.010390,0.900433,0.155844
01006,006 Pabellón de Arteaga,10769.0,1.569319,96.489925,0.362151,0.009286,1.457888,0.111431
01007,007 Rincón de Romos,12217.0,4.092658,93.975608,0.442007,0.024556,1.260539,0.204633
01011,011 San Francisco de los Romo,11702.0,1.222013,97.043240,0.726372,0.025637,0.734917,0.247821


## Guardado de datasets estándar

Los dataframes obtenidos a través de los procesos anteriores se guardan en libros de formato OpenXML (.xlsx) para facilitar su lectura tanto desde sistemas informáticos como por personas. Cada libro contiene 2 hojas:
1. Hoja de metadatos
2. Hoja de datos, con estimadores de las viviendas particulares habitadas y su distribución porcentual según:

HOJA | DESCRIPCION
--- | :---
02 | Material en pisos por municipio
08 | Combustible utilizado para cocinar por municipio
09 | Viviendas en las que sus ocupantes utilizan leña o carbón para cocinar y su distribución porcentual según disponibilidad de estufa o fogón con chimenea por municipio
19 | Forma de eliminación de residuos por municipio
20 | Viviendas en las que sus ocupantes entregan los residuos al servicio público de recolección o los colocan en un contenedor y su distribución porcentual
21 | Condición de separación y reutilización de residuos por municipio y forma de reutilización de los residuos
23 | Disponibilidad de equipamiento por municipio y tipo de equipamiento
25 | Fuente del abastecimiento del agua entubada por municipio
26 | Disponibilidad de drenaje y lugar de desalojo por municipio

Al ser datasets que provienen de una misma fuente, comparten varios campos de metadatos por lo que los campos en común se definen una sola vez y los campos particulares serán definidos a través de una función iterativa.

In [197]:
# Creacion de metadatos comunes
metadatos = {
    'Nombre del Dataset': 'Encuesta Intercensal 2015 - Tabulados de Vivienda',
    'Descripcion del dataset': np.nan,
    'Disponibilidad Temporal': '2015',
    'Periodo de actualizacion': 'No Determinada',
    'Nivel de Desagregacion': 'Municipal',
    'Notas': 'S/N',
    'Fuente': 'INEGI (Microdatos)',
    'URL_Fuente': 'http://www.beta.inegi.org.mx/proyectos/enchogares/especiales/intercensal/',
    'Dataset base': np.nan,
}



In [221]:
DSstandar['02'] = PisosDF

La función para la escritura de datasets estándar es el siguiente:

In [253]:
# Script para escritura de datasets estándar.
# La funcion espera los siguientes valores:
# --- hoja: (str) numero de hoja
# --- dataset: (Pandas DataFrame) datos que lleva la hoja
# --- metadatos: (dict) metadatos comunes para todas las hojas
# --- desc_hoja: (str) descripcion del contenido de la hoja

def escribedataset(hoja, dataset, metadatos, desc_hoja):
    # Compilación de la información
    datasetbaseurl = r'https://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/EI2015'
    directoriolocal = r'D:\PCCS\01_Dmine\Datasets\EI2015'
    archivo = hoja + '.xlsx'
    tempmeta = metadatos
    tempmeta['Descripcion del dataset'] = desc_hoja
    tempmeta['Dataset base'] = '"' + archivo + '" disponible en \n' + datasetbaseurl
    tempmeta = pd.DataFrame.from_dict(tempmeta, orient='index')
    tempmeta.columns = ['Descripcion']
    tempmeta = tempmeta.rename_axis('Metadato')
    
    # Escritura de dataset estándar
    destino = directoriolocal + '\\' + archivo
    writer = pd.ExcelWriter(destino)
    tempmeta.to_excel(writer, sheet_name ='METADATOS')
    dataset.to_excel(writer, sheet_name = hoja)
    writer.save()
    print('Se guardó: "{}" en \n{}'.format(desc_hoja, destino))

Una vez definida la función para la escritura de datasets, se ejecuta de manera iterativa sobre los datos:

In [254]:
for hoja, dataset in DSstandar.items():
    print('Procesando hoja '+hoja)
    escribedataset(hoja, dataset, metadatos, df.loc[hoja][0])

Procesando hoja 08
Se guardó: "Estimadores de las viviendas particulares habitadas y su distribución porcentual según combustible utilizado para cocinar por municipio" en 
D:\PCCS\01_Dmine\Datasets\EI2015\08.xlsx
Procesando hoja 09
Se guardó: "Estimadores de las viviendas particulares habitadas en las que sus ocupantes utilizan leña o carbón para cocinar y su distribución porcentual según disponibilidad de estufa o fogón con chimenea por municipio" en 
D:\PCCS\01_Dmine\Datasets\EI2015\09.xlsx
Procesando hoja 19
Se guardó: "Estimadores de las viviendas particulares habitadas y su distribución porcentual según la forma de eliminación de residuos por municipio" en 
D:\PCCS\01_Dmine\Datasets\EI2015\19.xlsx
Procesando hoja 20
Se guardó: "Estimadores de las viviendas particulares habitadas en las que sus ocupantes entregan los residuos al servicio público de recolección o los colocan en un contenedor y su distribución porcentual" en 
D:\PCCS\01_Dmine\Datasets\EI2015\20.xlsx
Procesando hoja 2

Al final del proceso quedaron se generaron 9 archivos:

In [256]:
for hoja in DSstandar.keys():
    print('**{}.xlsx**|{}'.format(hoja, df.loc[hoja][0]))

**08.xlsx**|Estimadores de las viviendas particulares habitadas y su distribución porcentual según combustible utilizado para cocinar por municipio
**09.xlsx**|Estimadores de las viviendas particulares habitadas en las que sus ocupantes utilizan leña o carbón para cocinar y su distribución porcentual según disponibilidad de estufa o fogón con chimenea por municipio
**19.xlsx**|Estimadores de las viviendas particulares habitadas y su distribución porcentual según la forma de eliminación de residuos por municipio
**20.xlsx**|Estimadores de las viviendas particulares habitadas en las que sus ocupantes entregan los residuos al servicio público de recolección o los colocan en un contenedor y su distribución porcentual
**21.xlsx**|Estimadores de las viviendas particulares habitadas y su distribución porcentual según condición de separación y reutilización de residuos por municipio y forma de reutilización de los residuos
**23.xlsx**|Estimadores de las viviendas particulares habitadas y su di